# QASVM Analysis Data Generation (Var=RealAmplitudes, N=8)

In [1]:
reps = 10
labels = (1, 2)

In [2]:
import sys
quic_path = '/home/quic/QUIC-Projects'
if not quic_path in sys.path:
    sys.path.append(quic_path)

In [3]:
import numpy as np 
from matplotlib import pyplot as plt

In [4]:
# for SVM
from classifiers.convex.svm import BinarySVM
from classifiers.kernel import Kernel
from classifiers.datasets.dataloader import Example_4x2, Sklearn_DataLoader
# for QASVM
from classifiers.quantum import Qasvm_Mapping_4x2
from classifiers.quantum.qasvm import QASVM
from classifiers.optimizer import SPSA
from qiskit.utils import QuantumInstance
from qiskit.ignis.mitigation import CompleteMeasFitter
from qiskit.circuit.library import RealAmplitudes, EfficientSU2, NLocal, TwoLocal
from qiskit.providers.aer import AerSimulator
from qiskit import IBMQ
from qiskit.circuit.library import PauliFeatureMap
# for logging & visualization
import logging
from classifiers.callback import CostParamStorage, ParamsStopping
from torch.utils.tensorboard import SummaryWriter
from classifiers.visualization import Plot_Data, Plot_Log_From_SPSA, Plot_SVM, compare_svm_and_qasvm

In [5]:
np.random.seed(13)
dl = Sklearn_DataLoader('iris', labels=labels)
X, y, _, _ = dl(8, true_hot=0)

In [6]:
svm = BinarySVM(Kernel('Pauli', reps=1), C=None, k=10, mutation='REDUCED_QASVM')
svm.fit(X, y)

In [7]:
var_form = RealAmplitudes(3, reps=reps)
feature_map = PauliFeatureMap(4, reps=1)
quantum_instance = QuantumInstance(AerSimulator(), shots=2**13)

In [8]:
qasvm = QASVM(X, y, 
              num_data_qubits=4, 
              var_form=var_form, 
              quantum_instance = quantum_instance, 
              feature_map = feature_map, 
              C=None, k=10, option='QASVM').dual

In [9]:
optimizer = SPSA(qasvm, blocking=True)

In [10]:
storage = CostParamStorage(interval=1)

In [ ]:
from tqdm.notebook import tqdm

epochs = 2**10
last_avg = 2**4

for epoch in tqdm(range(epochs)):
    optimizer.step(storage)
    if epoch>=last_avg and np.all(storage.last_avg(last_avg)<5e-2):
        break

qasvm.parameters = storage.last_avg(last_avg)

  0%|          | 0/1024 [00:00<?, ?it/s]

In [ ]:
regression = compare_svm_and_qasvm(svm, qasvm)

In [ ]:
qasvm.save(f'./models/qasvm(reps={reps}).pkl')
storage.save(f'./models/storage(reps={reps}).pkl')
np.save(f'./models/regression(reps={reps}).npy', regression)

In [ ]:
storage.plot_params()

In [ ]:
storage.plot()